Mount Google Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Imports

In [9]:
import tensorflow as tf
import numpy as np

# ================== PREPROCESS ==================
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mnv3_prep
from tensorflow.keras.applications.convnext import preprocess_input as convnext_prep
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as effv2_prep

In [10]:
# ================== CONFIG ==================
IMG_SIZE = 384  # MUST match training

# ✅ CORRECT CLASS ORDER (FROM TRAINING)
CLASS_NAMES = [
    "glioma",
    "meningioma",
    "no_tumor",
    "pituitary"
]


In [11]:
# ================== IMAGES TO TEST ==================
IMAGE_PATHS = {
    "glioma": "/content/drive/MyDrive/Project work/test_glioma.jpg",
    "meningioma": "/content/drive/MyDrive/Project work/test_meningioma.jpg",
    "pituitary": "/content/drive/MyDrive/Project work/test_pituitary.jpg",
    "no_tumor": "/content/drive/MyDrive/Project work/test_no tumor.jpg"
}

# ================== MODEL PATHS ==================
MODEL_PATHS = {
    "MobileNetV3": "/content/drive/MyDrive/Project work/models/tumor_mobilenet_v3.keras",
    "ConvNeXt-Tiny": "/content/drive/MyDrive/Project work/models/Tumor_convnext_tiny_tumor.keras",
    "EfficientNetV2-S": "/content/drive/MyDrive/Project work/models/Tumor_v2s_clean.keras",
}

In [12]:
# ================== IMAGE LOADER ==================
def load_image(path):
    img = tf.keras.utils.load_img(
        path,
        target_size=(IMG_SIZE, IMG_SIZE)
    )
    img = tf.keras.utils.img_to_array(img)
    img = tf.expand_dims(img, axis=0)
    return img

In [13]:
PREPROCESS = {
    "MobileNetV3": mnv3_prep,
    "ConvNeXt-Tiny": convnext_prep,
    "EfficientNetV2-S": effv2_prep,
}

In [14]:
# ================== LOAD MODELS ==================
models = {}

for name, path in MODEL_PATHS.items():
    print(f"Loading {name}...")
    models[name] = tf.keras.models.load_model(path)

print("All models loaded successfully.\n")

Loading MobileNetV3...
Loading ConvNeXt-Tiny...


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 96 variables whereas the saved optimizer has 190 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loading EfficientNetV2-S...
All models loaded successfully.



/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 232 variables whereas the saved optimizer has 462 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:

# ================== MANUAL TESTING ==================
print("🧠 Tumor Classification – Manual Image Testing")
print("=" * 60)

for true_label, img_path in IMAGE_PATHS.items():

    print(f"\nTesting image: {true_label}")
    print("-" * 60)

    img = load_image(img_path)

    for model_name, model in models.items():

        img_proc = PREPROCESS[model_name](img)
        preds = model.predict(img_proc, verbose=0)[0]

        idx = np.argmax(preds)
        pred_label = CLASS_NAMES[idx]
        confidence = preds[idx]

        print(
            f"{model_name:18s} → "
            f"Pred: {pred_label:12s} | "
            f"Conf: {confidence:.4f}"
        )


🧠 Tumor Classification – Manual Image Testing

Testing image: glioma
------------------------------------------------------------
MobileNetV3        → Pred: glioma       | Conf: 0.7437
ConvNeXt-Tiny      → Pred: glioma       | Conf: 0.4528
EfficientNetV2-S   → Pred: glioma       | Conf: 0.9657

Testing image: meningioma
------------------------------------------------------------
MobileNetV3        → Pred: meningioma   | Conf: 0.8019
ConvNeXt-Tiny      → Pred: meningioma   | Conf: 0.5716
EfficientNetV2-S   → Pred: meningioma   | Conf: 0.8944

Testing image: pituitary
------------------------------------------------------------
MobileNetV3        → Pred: meningioma   | Conf: 0.5737
ConvNeXt-Tiny      → Pred: pituitary    | Conf: 0.4762
EfficientNetV2-S   → Pred: pituitary    | Conf: 0.8575

Testing image: no_tumor
------------------------------------------------------------
MobileNetV3        → Pred: no_tumor     | Conf: 0.9156
ConvNeXt-Tiny      → Pred: no_tumor     | Conf: 0.9349
Effi